# NYCbuswatcher API Demo (against api.buswatcher.org)

In [25]:
# !pip install requests pandas geopandas ipyleaflet ipywidgets matplotlib python-dateutil

In [26]:
import requests
import pandas as pd
import geopandas as gpd

%matplotlib inline

In [27]:
# prevent cell wrapping in dataframe tables

In [28]:
%%html
<style>
.dataframe td {
    white-space: nowrap;
}
</style>

In [29]:
#-------------- timer decorator -------------------------------------------------------------
import functools
from time import perf_counter, strftime

def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = perf_counter()
        value = func(*args, **kwargs)
        toc = perf_counter()
        elapsed_time = toc - tic
        logging.warning(f"{func.__name__!r} finished at {strftime('%l:%M%p %Z on %b %d, %Y') } in {elapsed_time:0.4f} seconds")
        return value
    return wrapper_timer

## How to use the API:

A Shipment is a JSON file showing all observations of a single route for a single hour. 
There are several endpoints that let you:
- get a list of available Shipments to retrive
- retrieve the Shipments as JSON or GeoJOSN.

In [30]:
api_url = 'http://api.buswatcher.org/api/v2'

## Get a list of all the Shipments in history for a specific route

#### ENDPOINT: List All Shipments In History For Route: `/api/v2/nyc/{route}` 

In [31]:
route='M15'

In [32]:
shipments_for_route_url = f'{api_url}/nyc/{route}'
shipments_for_route_url

'http://api.buswatcher.org/api/v2/nyc/M15'

In [33]:
# fetch the index of shipments for this route
shipment_list = requests.get(shipments_for_route_url).json()

In [34]:
# inspect the response
list(shipment_list.keys())

['route', 'shipments']

In [35]:
# verify route
shipment_list['route']

'M15'

In [36]:
# how many shipment pointers in all
len(shipment_list['shipments'])

1053

In [37]:
# look at first record (shipments is a list of dicts, one for each shipment in the Data Store)
shipment_list['shipments'][0]

{'route': 'M15',
 'year': 2021,
 'month': 6,
 'day': 30,
 'hour': 23,
 'url': 'http://api.buswatcher.org:80/api/v2/nyc/2021/6/30/23/M15/buses'}

In [38]:
# and the last one
shipment_list['shipments'][-1]

{'route': 'M15',
 'year': 2021,
 'month': 8,
 'day': 25,
 'hour': 22,
 'url': 'http://api.buswatcher.org:80/api/v2/nyc/2021/8/25/22/M15/buses'}

#### grab one shipment... 

In [39]:
url = shipment_list['shipments'][0]['url']
shipment = requests.get(url).json()

#### look inside it... 

In [40]:
# how many buses?
len(shipment['buses'])

510

In [41]:
# look at one record
shipment['buses'][0]

{'route': 'M15',
 'timestamp': '2021-06-30 23:14:44-04:00',
 'route_long': 'MTA NYCT_M15',
 'direction': '0',
 'service_date': '2021-06-30',
 'trip_id': 'MTA NYCT_OH_C1-Weekday-134000_M15_243',
 'gtfs_shape_id': 'MTA_M150004',
 'route_short': 'M15',
 'agency': 'MTA NYCT',
 'origin_id': 'MTA_803019',
 'destination_name': 'EAST HARLEM 125 ST via 1 AV',
 'next_stop_id': 'MTA_401721',
 'next_stop_eta': '2021-06-30T23:15:12.240-04:00',
 'next_stop_d_along_route': 11470.57,
 'next_stop_d': 56.85,
 'lat': 40.784978,
 'lon': -73.943452,
 'bearing': 54.05072,
 'progress_rate': 'normalProgress',
 'vehicle_id': 'MTA NYCT_5881',
 'gtfs_block_id': 'MTA NYCT_OH_C1-Weekday_C_OH_13560_M15-206'}

In [42]:
# we can also get the shipment as geojson, jsut add /geojson to the end of the endpoint

url = shipment_list['shipments'][0]['url'] + '/geojson'
gj_shipment = requests.get(url).json()

In [43]:
# verify same # of buses
len(gj_shipment['features'])

510

In [44]:
# and check out the geojson (hint: you can paste this into geojson.io to make maps instantly)
gj_shipment['features'][0]

{'type': 'Feature',
 'properties': {'route': 'M15',
  'timestamp': '2021-06-30 23:14:44-04:00',
  'route_long': 'MTA NYCT_M15',
  'direction': '0',
  'service_date': '2021-06-30',
  'trip_id': 'MTA NYCT_OH_C1-Weekday-134000_M15_243',
  'gtfs_shape_id': 'MTA_M150004',
  'route_short': 'M15',
  'agency': 'MTA NYCT',
  'origin_id': 'MTA_803019',
  'destination_name': 'EAST HARLEM 125 ST via 1 AV',
  'next_stop_id': 'MTA_401721',
  'next_stop_eta': '2021-06-30T23:15:12.240-04:00',
  'next_stop_d_along_route': 11470.57,
  'next_stop_d': 56.85,
  'lat': 40.784978,
  'lon': -73.943452,
  'bearing': 54.05072,
  'progress_rate': 'normalProgress',
  'vehicle_id': 'MTA NYCT_5881',
  'gtfs_block_id': 'MTA NYCT_OH_C1-Weekday_C_OH_13560_M15-206'},
 'geometry': {'type': 'Point', 'coordinates': [-73.943452, 40.784978]}}

#### ...and map it

In [67]:
from ipywidgets import Layout
from ipyleaflet import (
    Map, basemaps, basemap_to_tiles,
    Circle, Marker, Rectangle, LayerGroup
)

# init the data layer
buses_gdf = gpd.GeoDataFrame.from_features(gj_shipment['features'])

# init the map
defaultLayout=Layout(width='960px', height='540px')
center = (40.7128, -74.0060) #reverse?
zoom = 12


toner = basemap_to_tiles(basemaps.Stamen.Toner)
m = Map(layers=(toner, ), center=center, zoom=zoom, layout=defaultLayout)

# Create layer group
layer_group = LayerGroup()


for index, row in buses_gdf.iterrows():
    marker = Marker(location=(row.lat, row.lon), draggable=False)
    layer_group.add_layer(marker);


m.add_layer(layer_group)

m

Map(center=[40.7128, -74.006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

## build a route history 

#### (e.g. all observations ever made for a single route)
#### by iterating over list of shipments and load them all into a dataframe (takes a few minutes)

In [ ]:
def get_route_history(route):
    
    # get the list of shipments for a route
    shipments_for_route_url = f'{api_url}/nyc/{route}'
    shipment_list = requests.get(shipments_for_route_url).json()
    
    # init list to hold results of the individual fetches
    rows=[]

    # iterate over the list of shipments and get each one
    for s in shipment_list['shipments']:
        shipment = requests.get(s['url']).json()
        for bus_dict in shipment['buses']:
            rows.append(bus_dict)
        
    df = pd.DataFrame.from_dict(rows, orient='columns')
    df['passenger_count'] = df['passenger_count'].fillna(0)
    print(f'loaded {len(rows)} buses from {len(shipment_list["shipments"])} shipments for route {route} into DataFrame: "df"')
    
    return df

df = get_route_history(route)
df